In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from random import shuffle

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

#testy

In [ ]:
df = pd.read_csv("./clean_data/fully_merged_data.csv")
df

In [ ]:
# delete empty values
df = df.dropna()
df

In [ ]:
# remove null values
arr = df.index[df["fel_misd"] == ' ']
df = df.drop(arr, axis=0)
arr = df.index[df["fel_misd"] == '\xa0']
df = df.drop(arr, axis=0)
df

In [ ]:
# z-score normalize desired columns
from scipy.stats import zscore

need_norm = ["age","MEDHINC_CY", "WLTHINDXCY", "TOTHH_CY"]
norm = df[need_norm].apply(zscore)
norm

In [ ]:
df[need_norm] = norm
df

In [ ]:
# removing the imbalanced classes
# temp1 = df[df['fel_misd'] == 2.0]
# temp2 = df[df['fel_misd'] == 1.0]
# df = pd.concat([temp1, temp2])
# df = df.drop(temp)
# temp = df.index[df['fel_misd'] == 1.0].tolist()
# df = df.drop(temp)

In [ ]:
df["fel_misd"].value_counts()

In [ ]:
df_pandas_encoded = pd.get_dummies(df, columns=['sex', 'day', 'month'], drop_first=True)
# df_pandas_encoded = pd.get_dummies(df_pandas_encoded, columns=["fel_misd"], drop_first=False)
df_pandas_encoded.head()

In [ ]:
m_arr = df_pandas_encoded.index[df_pandas_encoded["fel_misd"] == 'M'].tolist()
f_arr = df_pandas_encoded.index[df_pandas_encoded["fel_misd"] == 'F'].tolist()

shuffle(m_arr)
shuffle(f_arr)

In [ ]:
df_pandas_encoded = df_pandas_encoded.drop(m_arr[0:44000], axis = 0)
df_pandas_encoded = df_pandas_encoded.drop(f_arr[0:14000], axis = 0)
df_pandas_encoded['fel_misd'].value_counts()

In [ ]:
df_pandas_encoded = df_pandas_encoded.drop("WLTHINDXCY", axis=1)

In [ ]:
inp = list(df_pandas_encoded.columns)
# oup = ['fel_misd_0.0',
#      'fel_misd_1.0',
#      'fel_misd_2.0',
#      'fel_misd_3.0',
#      'fel_misd_4.0',
#      'fel_misd_5.0']
oup = ["fel_misd"]
for x in oup:
    inp.remove(x)
inp, oup

In [ ]:
x,y = df_pandas_encoded[inp], df_pandas_encoded[oup]
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, test_size=0.2)

In [ ]:
import itertools
items = ['MEDHINC_CY','age','sex_M', 'day', 'month']
combs = []
for i in range(1, len(items)):
    combs.append(list(set(itertools.combinations(items, i))))

In [ ]:
day_vals = ['day_1',
          'day_2',
          'day_3',
          'day_4',
          'day_5',
          'day_6']

month_vals = ['month_1',
              'month_2',
              'month_3',
              'month_4',
              'month_5',
              'month_6',
              'month_7',
              'month_8',
              'month_9',
              'month_10',
              'month_11']
fel_misd_vals = [ 'fel_misd_1',
                  'fel_misd_2',
                  'fel_misd_3',
                  'fel_misd_4',
                  'fel_misd_5']

best_dict = {"features": [], "score": -2**31}
for k_amt in combs:
    for ind_comb in k_amt:
        comb = list(ind_comb)
        if 'day' in comb:
            comb.remove('day')
            comb += day_vals
        if 'month' in comb:
            comb.remove('month')
            comb += month_vals
        if 'fel_misd' in comb:
            comb.remove('fel_misd')
            comb += fel_misd_vals
        
        x_subset = x_train[comb].values
        cvs = cross_val_score(linear_model.LogisticRegression(multi_class='ovr'), x_subset, y_train)
        
        if cvs.mean() > best_dict["score"]:
            best_dict["features"] = comb
            best_dict["score"] = cvs.mean()
best_dict

In [ ]:
lr = linear_model.LogisticRegression(multi_class="ovr")

In [ ]:
best_dict["features"]

In [ ]:
x_train = x_train[best_dict["features"]]
x_test = x_test[best_dict["features"]]

lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)
print(f"Accuracy train: {lr.score(x_train,y_train)}")
print(f"Accuracy Test: {lr.score(x_test,y_test)}")

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))